In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator
import spacy
import numpy as np
import random
import math
import time
from nltk.translate.bleu_score import sentence_bleu

In [0]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [0]:
!python -m spacy download en
!python -m spacy download de

spacy_en = spacy.load('en')
spacy_de = spacy.load('de')

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

    
sourceLanguage = Field( tokenize = tokenize_en, 
                        init_token = '<sos>', 
                        eos_token = '<eos>')

targetLanguage = Field( tokenize = tokenize_de, 
                        init_token = '<sos>', 
                        eos_token = '<eos>')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [0]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.en', '.de'), fields = (sourceLanguage, targetLanguage))
sourceLanguage.build_vocab(train_data, min_freq = 2)
targetLanguage.build_vocab(train_data, min_freq = 2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 687kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 236kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 226kB/s]


In [0]:
BATCH_SIZE = 1
INPUT_DIM  = len(sourceLanguage.vocab) # Kx
OUTPUT_DIM = len(targetLanguage.vocab) # Ky
EMB_DIM = 256 # m 
HID_DIM = 512 # n
MAXOUT_DIM = 400 # l 
ATT_HID_DIM = 1000 # n'

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, device):
        super(EncoderRNN, self).__ini__()
        self.embedding_dim  = embedding_dim
        self.embedding      = nn.Embedding(input_dim, embedding_dim)
        self.rnn            = nn.GRU(embedding_dim, embedding_dim)

    def forward(self, input, hidden):
        output          = self.embedding(input).view(1, 1, -1)
        output, hidden  = self.rnn(emb, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=self.device)

In [0]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_dim, output_dim, device):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_dim, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim)
        self.fully_connected = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = nn.GRU(output, hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=self.device)

In [0]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, 
          decoder_optimizer, criterion, max_length=20):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length,
                                  encoder.hidden_size, 
                                  device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach() 

        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == EOS_token:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [0]:
def beam_search(decoder, encoder_hidden, beam_width=3, target_lenght):
    hypotheses = [(1.0, [SOS_token], encoder_hidden)]
    for i in range(1, target_length):
        for hypothesis in hypotheses:
            score, seq, hidden = hypothesis
            decoder_output, decoder_hidden = decoder(seq[-1], hidden)
            prob, label = decoder_output.topk(beam_width)
            hypotheses += [(score*prob[i], seq + label, decoder_hidden) for i in range(beam_width)]
        hypotheses = sorted(hypotheses)[:5]
    _, seq, _ = max(hypotheses)
    return seq[1:-1]